In [1]:
import os
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler

In [2]:
train = pd.DataFrame()
for name in os.listdir('proc_data/total')[:-1]:
    ins_data = pd.read_csv(f'proc_data/total/{name}', index_col=0, parse_dates=['date'])
    train = pd.concat([train, ins_data], axis=0).reset_index(drop=True)
    
train.sample(5)

,Np,Tp,Vp,B_gsm_x,B_gsm_y,B_gsm_z,Bmag,target
19390,-291.311935,69191.068047,92.614793,3.907982,-0.143746,-0.632663,4.031828,2
37409,-9999.900000,21246.265680,-194.204852,1.763716,-1.292107,-2.094615,3.885544,2
12788,-404.466385,41241.689941,-22.210414,-1.671219,3.374426,2.533550,4.788722,3
31107,-352.465337,31213.506509,11.574083,3.307964,-1.568882,-0.178124,3.770817,0
31453,-9999.900000,-9999.900000,-1689.102722,0.252308,-1.776112,1.053822,2.785515,0


In [3]:
test = pd.read_csv(f'proc_data/total/{os.listdir("proc_data/total")[-1]}', index_col=0, parse_dates=['date'])

test.sample(5)

,Np,Tp,Vp,B_gsm_x,B_gsm_y,B_gsm_z,Bmag,target
date,,,,,,,,
2013-02-03 21:00:00,-7855.808411,51341.137500,-277.378095,1.253851,-3.583863,0.179649,4.837589,1
2013-12-15 06:00:00,-530.334060,100818.444643,32.389940,2.209685,-4.103821,1.368315,5.752262,1
2013-01-05 21:00:00,-9999.900000,-9999.900000,-4088.537633,-3.517775,2.605367,0.443917,4.679692,0
2013-02-21 09:00:00,-9999.900000,58731.535119,-25.586190,3.764988,-2.529554,2.393911,7.206762,1
2013-03-14 21:00:00,-9999.900000,60959.760119,-2215.281369,-5.026804,4.289155,4.593958,9.659321,1


In [4]:
problem = pd.read_csv("proc_data/problem.csv", index_col=0, parse_dates=['dt'])

problem.head()

,Np,Tp,Vp,B_gsm_x,B_gsm_y,B_gsm_z,Bmag
dt,,,,,,,
2015-01-01 00:00:00,1.852,41618.0,455.61,6.098,2.449,-13.415,14.952
2015-01-01 00:01:00,2.181,33781.0,461.07,6.700,1.502,-13.232,14.912
2015-01-01 00:02:00,2.415,34709.0,464.84,7.390,0.523,-12.941,14.928
2015-01-01 00:03:00,2.359,34183.0,470.76,8.011,-0.098,-12.694,15.013
2015-01-01 00:04:00,1.237,32467.0,469.51,7.493,0.410,-13.058,15.064


In [5]:
mmscler = MinMaxScaler()
train_x, train_y = mmscler.fit_transform(train.iloc[:, :-1].values)[:, :, np.newaxis], train.target.values
test_x, test_y = mmscler.fit_transform(test.iloc[:, :-1].values)[:, :, np.newaxis], test.target.values
problem__mm = mmscler.fit_transform(problem.values)[:, :, np.newaxis]
print(train_x.shape, test_x.shape)

(40912, 7, 1) (2920, 7, 1)


In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

tf.debugging.set_log_device_placement(True)
tf.__version__

'2.0.0-rc0'

In [7]:
optim = keras.optimizers.RMSprop()
loss_fn = keras.losses.MeanSquaredError()

model = keras.models.Sequential()
model.add(layers.GRU(128, input_shape=(None, 1)))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(1, activation='linear'))
model.compile(optimizer=optim, loss=loss_fn, metrics=['acc'])
model.summary()

Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RandomStandardNormal in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Qr in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op DiagPart in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sign in device /job:localhost/replica:0/task:0

In [8]:
early_stop = EarlyStopping(monitor='loss', patience=3, verbose=1)

history = model.fit(train_x, train_y, 32, epochs=100, callbacks=[early_stop])
history.params

Executing op RangeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op TensorDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ZipDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op DatasetCardinality in device /job:localhost/replica:0/task:0/device:CPU:0
Train on 40912 samples
Epoch 1/100
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AnonymousIteratorV2 in device /job:localhost/

UnknownError:  [_Derived_]  Fail to find the dnn implementation.
	 [[{{node CudnnRNN}}]]
	 [[sequential/gru/StatefulPartitionedCall]] [Op:__inference_distributed_function_3239]

Function call stack:
distributed_function -> distributed_function -> distributed_function


In [ ]:
model.evaluate(test_x, test_y)

In [ ]:
model.predict(problem__mm)